In [1]:
from reggie.ingestion.utils import format_column_name
from reggie.configs.configs import Config
import logging
import pandas as pd
import numpy as np
import datetime
from io import StringIO
from datetime import datetime
import time
import numpy as np
from functools import reduce
from collections import defaultdict, OrderedDict
from dateutil import parser
import json
%load_ext line_profiler
import os

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [2]:
entries = os.listdir('/home/tommi/Downloads/OK 2021-03-06/OK 2021-03-06/CTYSW_VR_20210308110648')

In [3]:
vdf = pd.DataFrame()

In [4]:
dtypes = {'MuniSub': 'float64'}

In [5]:
for file in entries:
    if 'CTY' in file:
        temp_df = pd.read_csv('/home/tommi/Downloads/OK 2021-03-06/OK 2021-03-06/CTYSW_VR_20210308110648/' + file, encoding='latin', low_memory=False, dtype=dtypes)
        vdf = pd.concat([vdf, temp_df], ignore_index=True)


In [6]:
# vdf = pd.read_csv('/home/tommi/Downloads/OK 2021-03-06/OK 2021-03-06/CTYSW_VR_20210308110648/CTY01_vr.csv')
precincts = pd.read_csv('/home/tommi/Downloads/OK 2021-03-06/OK 2021-03-06/CTYSW_VR_20210308110648/precincts.csv')
# vdf2 = pd.read_csv('/home/tommi/Downloads/OK 2021-03-06/OK 2021-03-06/CTYSW_VR_20210308110648/CTY02_vr.csv')

In [7]:
vdf.columns

Index(['Precinct', 'LastName', 'FirstName', 'MiddleName', 'Suffix', 'VoterID',
       'PolitalAff', 'Status', 'StreetNum', 'StreetDir', 'StreetName',
       'StreetType', 'StreetPostDir', 'BldgNum', 'City', 'Zip', 'DateOfBirth',
       'OriginalRegistration', 'MailStreet1', 'MailStreet2', 'MailCity',
       'MailState', 'MailZip', 'Muni', 'MuniSub', 'School', 'SchoolSub',
       'TechCenter', 'TechCenterSub', 'CountyComm', 'VoterHist1',
       'HistMethod1', 'VoterHist2', 'HistMethod2', 'VoterHist3', 'HistMethod3',
       'VoterHist4', 'HistMethod4', 'VoterHist5', 'HistMethod5', 'VoterHist6',
       'HistMethod6', 'VoterHist7', 'HistMethod7', 'VoterHist8', 'HistMethod8',
       'VoterHist9', 'HistMethod9', 'VoterHist10', 'HistMethod10'],
      dtype='object')

In [8]:
precincts.rename(columns={"PrecinctCode": "Precinct"}, inplace=True)

# This is an attempt to make a dict, but creating multiple columns based on a dict lookup might not be worth it
# as it requires use of the map feature, merging at the end might just be faster. OK isn't huge anyway
# precincts.set_index("Precinct", inplace=True)
# precinct_dict = precincts.to_dict('Index')

In [9]:
unholy_union = vdf.merge(precincts, how='left', on='Precinct')

In [10]:
unholy_union.columns

Index(['Precinct', 'LastName', 'FirstName', 'MiddleName', 'Suffix', 'VoterID',
       'PolitalAff', 'Status', 'StreetNum', 'StreetDir', 'StreetName',
       'StreetType', 'StreetPostDir', 'BldgNum', 'City', 'Zip', 'DateOfBirth',
       'OriginalRegistration', 'MailStreet1', 'MailStreet2', 'MailCity',
       'MailState', 'MailZip', 'Muni', 'MuniSub', 'School', 'SchoolSub',
       'TechCenter', 'TechCenterSub', 'CountyComm', 'VoterHist1',
       'HistMethod1', 'VoterHist2', 'HistMethod2', 'VoterHist3', 'HistMethod3',
       'VoterHist4', 'HistMethod4', 'VoterHist5', 'HistMethod5', 'VoterHist6',
       'HistMethod6', 'VoterHist7', 'HistMethod7', 'VoterHist8', 'HistMethod8',
       'VoterHist9', 'HistMethod9', 'VoterHist10', 'HistMethod10',
       'CongressionalDistrict', 'StateSenateDistrict', 'StateHouseDistrict',
       'CountyCommissioner', 'PollSite'],
      dtype='object')

In [11]:
unholy_union['PolitalAff'].isnull().values.any()

False

In [12]:
unholy_union["VoterID"]
for x in unholy_union.columns:
    print(x, unholy_union[x].apply(type).unique(), unholy_union[x].isnull().values.any())

Precinct [<class 'int'>] False
LastName [<class 'str'> <class 'float'>] True
FirstName [<class 'str'> <class 'float'>] True
MiddleName [<class 'str'> <class 'float'>] True
Suffix [<class 'float'> <class 'str'>] True
VoterID [<class 'int'>] False
PolitalAff [<class 'str'>] False
Status [<class 'str'>] False
StreetNum [<class 'str'> <class 'float'> <class 'int'>] True
StreetDir [<class 'str'> <class 'float'>] True
StreetName [<class 'str'> <class 'float'>] True
StreetType [<class 'str'> <class 'float'>] True
StreetPostDir [<class 'float'> <class 'str'>] True
BldgNum [<class 'str'> <class 'float'>] True
City [<class 'str'> <class 'float'>] True
Zip [<class 'str'> <class 'float'>] True
DateOfBirth [<class 'str'> <class 'float'>] True
OriginalRegistration [<class 'str'> <class 'float'>] True
MailStreet1 [<class 'str'> <class 'float'>] True
MailStreet2 [<class 'float'> <class 'str'>] True
MailCity [<class 'str'> <class 'float'>] True
MailState [<class 'str'> <class 'float'>] True
MailZip [<c

In [14]:
unholy_union['VoterHist1'].unique()

array(['11/03/2020', '06/26/2018', nan, '02/11/2020', '11/06/2018',
       '11/08/2016', '02/12/2019', '06/30/2020', '04/02/2019',
       '03/03/2020', '11/06/2012', '11/12/2013', '08/26/2014',
       '02/14/2017', '04/03/2012', '06/24/2014', '11/04/2014',
       '02/12/2013', '03/01/2016', '03/03/2015', '04/04/2017',
       '02/13/2018', '06/28/2016', '04/02/2013', '04/01/2014',
       '11/07/2006', '07/27/2004', '08/28/2018', '10/13/2009',
       '02/09/2016', '11/04/2008', '11/02/2010', '06/26/2012',
       '08/23/2016', '09/10/2019', '01/08/2019', '05/09/2006',
       '02/10/2015', '10/08/2013', '02/11/2014', '08/22/2006',
       '08/25/2020', '11/14/2017', '08/27/2002', '05/14/2013',
       '05/10/2011', '04/03/2018', '08/28/2012', '11/02/2004',
       '02/10/2009', '08/11/2015', '03/06/2018', '07/14/2015',
       '04/07/2015', '11/07/2000', '02/09/2010', '07/27/2010',
       '09/08/2009', '06/11/2013', '03/02/2021', '04/05/2016',
       '02/09/2021', '06/09/2015', '11/10/2015', '